- Using the pre-build RAPIDS image on Google Cloud's AI Platform Notebooks with a T4 GPU, 4vCPUs, 15GB RAM
- https://cloud.google.com/ai-platform/notebooks/docs/images#deciding
- This should provide CUDA 10.0, rapids 0.12

In [ ]:
%%bash
nvidia-smi
nvcc --version

In [ ]:
import numpy as np; print('numpy Version:', np.__version__)
import pandas as pd; print('pandas Version:', pd.__version__)
import xgboost as xgb; print('XGBoost Version:', xgb.__version__)
import cudf; print('cudf Version:', cudf.__version__)
import cuml; print('cudf Version:', cuml.__version__)
import time

Download HIGGs dataset & unzip
https://archive.ics.uci.edu/ml/datasets/HIGGS

In [ ]:
# %%bash
# wget https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz -P /home/jupyter/

## Single Node with CPUs (PANDAS + XGBoost) or single GPU (RAPIDS-cuDF + XGBoost)
- Read dataset: cudf.read_csv reads dataset directly into GPU using cudf/rapids
- pd.read_csv takes ~80s, cudf.read_csv takes ~35s
- XGBoost w/ RAPIDS examples https://rapids.ai/xgboost.html

In [ ]:
def xgboost_fun(gpu_cpu, tree_method):
    colnames = ['label'] + ['feature-%02d' % i for i in range(1, 29)]
    start_time = time.time()
    if gpu_cpu=='cpu':
        df=pd.read_csv('/home/jupyter/HIGGS.csv', header=None, names=colnames)
    else: 
        df=cudf.read_csv('/home/jupyter/HIGGS.csv', header=None, names=colnames)
    print("[INFO]: ------ Data Ingestion is completed in {} seconds ---".format((time.time() - start_time)))

    X = df[df.columns.difference(['label'])]
    y = df['label']
    dtrain=xgb.DMatrix(X,y)
    param =  {
               'max_depth': 8,
               'objective': 'reg:squarederror',
               'tree_method': tree_method
             }

    start_time = time.time()
    bst = xgb.train(param, dtrain)
    print("[INFO]: ------ Training is completed in {} seconds ---".format((time.time() - start_time)))
    return bst

In [ ]:
bst=xgboost_fun('gpu','gpu_hist')
# bst=xgboost_fun('cpu','hist')

In [ ]:
bst=xgboost_fun('cpu','hist')

## TODO: Single Node with multiple GPUS (Dask + RAPIDS)


## TODO: Multi-Node with multiple GPUS (Dask + RAPIDS)